
# 4-4.scikit-learn(8:20)


# 1:前処理

* 前処理は大事だよ
* データ解析の8割から9割が前処理だよ
* scikit-learnはクラスとインスタンス化してfitメソッドとtransformメソッドで前処理を行えるよ

## 欠損値への対応

* データの収集時に測定できてなかったとか保存できてなかったとかで値が欠損してるやつ
* 欠損値をそのままにすると解析が難しくなるから適切に対処するのが大事だよ
* 対応方法は主に2つに分けられるよ
    * 欠損値を除去する
    * 欠損値を保管する

In [1]:
import numpy as np
import pandas as pd

# サンプルのデータセット作成

df = pd.DataFrame(
    {
        'A': [1, np.nan, 3, 4, 5],
        'B': [6, 7, 8, np.nan, 10],
        'C': [11, 12, 13, 14, 15]
    }
)
df

,A,B,C
0,1.0,6.0,11
1,NaN,7.0,12
2,3.0,8.0,13
3,4.0,NaN,14
4,5.0,10.0,15


In [9]:
### 欠損値の削除

In [6]:
df.isnull() # 欠損値の確認

,A,B,C
0,False,False,False
1,True,False,False
2,False,False,False
3,False,True,False
4,False,False,False


In [7]:
df.dropna() # 欠損値がある行を削除

,A,B,C
0,1.0,6.0,11
2,3.0,8.0,13
4,5.0,10.0,15


### 欠損値の補完
* 平均値とか、中央地とか、最頻値とかを代入するよ
* pandas側で処理するならfillnaメソッドで補完するよ
* scikit-learnならImputerクラスで補完するよ

In [17]:
from sklearn.impute import SimpleImputer

# 平均値で欠損値を補完するためのインスタンスを作成する
imp = SimpleImputer(strategy = 'mean')
# 欠損値を補完
imp.fit(df)dd
imp.transform(df)

array([[ 1.  ,  6.  , 11.  ],
       [ 3.25,  7.  , 12.  ],
       [ 3.  ,  8.  , 13.  ],
       [ 4.  ,  7.75, 14.  ],
       [ 5.  , 10.  , 15.  ]])

## カテゴリ変数のエンコーディング
* 血液型とかね、職業とかね、いくつかの限られた値において、どれに該当しているのかを示す変数だよ
* 対応方法は主に2つに分けられるよ

### カテゴリ変数のエンコーディング
* a→0,b→1,c→2…っていう形でエンコーディングするよ

In [28]:
#下準備
import pandas as pd
df = pd.DataFrame(
    {
        'A' : [1,2,3,4,5],
        'B' : ['a','b','a','b','c']
    }
)
df

,A,B
0,1,a
1,2,b
2,3,a
3,4,b
4,5,c


In [29]:
from sklearn.preprocessing import LabelEncoder

# カテゴリ変数のエンコーディング

le = LabelEncoder() # ラベルエンコーダのインスタンスを生成
le.fit(df['B']) # ラベルのエンコーディング
le.transform(df['B'])

array([0, 1, 0, 1, 2])

In [30]:
# 元の値を確認
le.classes_

array(['a', 'b', 'c'], dtype=object)

### One-hotエンコーディング
* 符号化の処理
* 取りうる値の文だけ列を増やして、各業の該当する値の列のみに1が、それ以外の列には0が入力されるように変換するよ

In [35]:
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer

# DataFrameをコピー
df_ohe = df.copy()
print(df_ohe)

# ColumnTransformerのインスタンス化
ct = ColumnTransformer([('B_x', CountVectorizer(analyzer=lambda x: [x]), 'B')],
    remainder = 'passthrough')
ct.fit_transform(df_ohe)

   A  B
0  1  a
1  2  b
2  3  a
3  4  b
4  5  c


array([[1, 0, 0, 1],
       [0, 1, 0, 2],
       [1, 0, 0, 3],
       [0, 1, 0, 4],
       [0, 0, 1, 5]])

## 特徴量の正規化
* 特徴量Aが0~99の数値、特徴量Bが0~10000の数値だったら、特徴量Bの影響を大きく受けるよね。こまるー。
* 特徴量Aのと特徴量Bのオーダーが同様になるように尺度を揃える必要があるよね。←これが正規化
* とりあえずここでは2種類学ぶよ
    * 分散正規化
    * 最小最大正規化

### 分散正規化

* 特徴量の平均が0、標準偏差が1となるように特徴量を変換するよ。
* 数式で書くと、

$$
x' = \frac{x-\mu}{\sigma}
$$
* $x$：特徴量
* $x'$：分散正規化された特徴量
* $\mu$：特徴量の平均
* $\sigma$：標準偏差
* StandardScalerクラスを使うよ

In [39]:
# 分散正規化用のサンプル作成
import pandas as pd
df = pd.DataFrame(
    {
        'A':[1,2,3,4,5],
        'B':[100,200,400,500,800]
    }
)
df

,A,B
0,1,100
1,2,200
2,3,400
3,4,500
4,5,800


In [44]:
# 列Aの平均:(1+2+3+4+5)/5=3
print('列Aの平均：',df['A'].mean())
# 列Aの標準偏差:1.41421356...
print('列Aの標準偏差:',df['A'].std(ddof=0))
# 列Bの平均:400
print('列Bの平均：',df['B'].mean())
# 列Aの標準偏差:244.94897428...
print('列Bの標準偏差:',df['B'].std(ddof=0))

列Aの平均： 3.0
列Aの標準偏差: 1.4142135623730951
列Bの平均： 400.0
列Bの標準偏差: 244.94897427831782


In [46]:
# standardscalerクラスを使う例
from sklearn.preprocessing import StandardScaler
# 分散正規化のインスタンスを生成
stdsc = StandardScaler()
# 分散正規化を実行
stdsc.fit(df)
stdsc.transform(df)

array([[-1.41421356, -1.22474487],
       [-0.70710678, -0.81649658],
       [ 0.        ,  0.        ],
       [ 0.70710678,  0.40824829],
       [ 1.41421356,  1.63299316]])

### 最小最大正規化
* 特徴量の最小値が0、最大値が1を取るように特徴量を正規化するよ
* 数式で書くとこうなるよ

$$
x' = \frac{x-x_{min}}{x_{max}-x_{min}}
$$
* $x$:特徴量
* $x'$:最小最大正規化された特徴量
* $x_{min}$:特徴量の最小値
* $x_{max}$:特徴量の最大値

# 2:分類

## 分類モデル構築の流れ

## サポートベクタマシン

## 決定木

## ランダムフォレスト

# 3:回帰

# 4:次元削減

## 主成分分析

# 5:モデルの評価

## カテゴリの分類精度

## 予測確率の正確さ

# 6:ハイパーパラメータの最適化

# 7:クラスタリング

## k-means

## 階層的クラスタリング